In [4]:
!git clone -q https://github.com/aryanda1/voimo-backend.git
!mv /content/voimo-backend/* /content/
!rm -rf voimo-backend

In [5]:
import environnew

Setting up environment...

Environment set up in 361 seconds


In [6]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone -q https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr -q
!pip install facexlib -q
!pip install gfpgan -q
!pip install -r requirements.txt -q
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
%cd ..

/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 99.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.9.1 requires google-auth-oauthlib<0.5,>=0.4.1, but you have google-auth-oauthlib 1.0.0 which is incompatible.
tensorboard 2.9.1 requires tensorboard-data-server<0.7.0,>=0.6.0, but you have tensorboard-data-server 0.7.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [7]:
openai_api = input('Enter openai Api key')

Enter openai Api keysk-7T0wgDrCLxvc4gqVHfzQT3BlbkFJ9oXiOyfphibkfcxbXzrK


In [15]:
import gdown
gdown.download('https://drive.google.com/uc?export=download&id=12L9Z5fO748i8Ua9q0qZBhwwBJ0SyM0rg','models/model.ckpt')

Downloading...
From: https://drive.google.com/uc?export=download&id=12L9Z5fO748i8Ua9q0qZBhwwBJ0SyM0rg
To: /content/models/model.ckpt
100%|██████████| 2.13G/2.13G [00:56<00:00, 38.0MB/s]


'models/model.ckpt'

In [16]:
from fastapi import FastAPI, Request, File, Form, BackgroundTasks
from fastapi.responses import FileResponse
from fastapi.middleware.cors import CORSMiddleware
import sys
import os
sys.path.extend([
    'src/taming-transformers',
    'src/clip',
    'stable-diffusion/',
    'k-diffusion',
    'pytorch3d-lite',
    'AdaBins',
    'MiDaS',
])
import load_model
import torch
from mainOld import musictoVid
import time
app = FastAPI()
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

custom_checkpoint_path = 'models/model.ckpt'
models_path = "/content/models"
os.makedirs(models_path, exist_ok=True)
model = load_model.model(models_path,custom_checkpoint_path=custom_checkpoint_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

tasks = {}

@app.get('/')
async def home():
  return {'mes':'Welcome!'}


def long_running_task(device, model, audio_path, theme, artist, option, upscale, task_id):
    # Do the long-running task here
    video_path = musictoVid(device, model, audio_path, theme, artist, int(option), upscale,task_id,openai_api)
    tasks[task_id]['status'] = 'completed'
    tasks[task_id]['path'] = video_path

@app.post("/submit_form")
async def submit_form(background_tasks: BackgroundTasks,audio: bytes = File(...), artist: str = Form(''), theme: str = Form(''), option: str = Form(...), upscale: bool = Form(False)):
    # Save audio to current directory
    with open("audio.wav", "wb") as f:
        f.write(audio)

    # Print other form inputs
    # print(f"Text input 1: {theme}")
    # print(f"Text input 2: {artist}")
    # print(f"Radio button: {option}")
    # print(f"Checkbox: {upscale}")
    timestring = time.strftime('%Y%m%d%H%M%S')
    tasks[timestring] = {'status':'processing'}
    background_tasks.add_task(long_running_task, device, model, '/content/audio.wav', theme, artist, option, upscale, timestring)
    # return FileResponse(video_path)
    return {"id": timestring}

@app.post("/get_video/{task_id}")
async def get_video(task_id: str):
    # Retrieve the result of the task from the task dictionary using the task ID
    if task_id in tasks:
        task = tasks[task_id]
        status = task["status"]
        if status == "completed":
            video_path = task["path"]
            print(video_path)
            filename = video_path.split('/')[-1]
            headers = {"Content-Disposition": f'attachment; filename="{filename}"'}
            return FileResponse(video_path, headers=headers)
        else:
            return {"status": status}
    else:
        return {"error": "Task not found"}

Using config: ./stable-diffusion/configs/stable-diffusion/v1-inference.yaml
Using ckpt: models/model.ckpt
Loading model from models/model.ckpt
Global Step: None
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.11.self_attn.v_proj.weight', 'vision_model.encoder.layers.16.self_attn.v_proj.bias', 'vision_model.encoder.layers.1.self_attn.q_proj.bias', 'vision_model.encoder.layers.1.layer_norm1.bias', 'vision_model.encoder.layers.18.self_attn.k_proj.bias', 'vision_model.encoder.layers.20.mlp.fc2.bias', 'vision_model.encoder.layers.3.layer_norm2.weight', 'vision_model.embeddings.patch_embedding.weight', 'vision_model.encoder.layers.23.self_attn.k_proj.bias', 'vision_model.encoder.layers.15.mlp.fc2.weight', 'vision_model.encoder.layers.13.self_attn.out_proj.bias', 'vision_model.encoder.layers.11.self_attn.q_proj.bias', 'vision_model.encoder.layers.14.mlp.fc1.weight', 'vision_model.encoder.layers.15.mlp.fc2.bias', 'vision_model.encoder.layers.1.self_attn.k_proj.weight', 'vision_model.encoder.layers.21.mlp.fc1.bias', 'vision_model.encoder.layers.10.self_at

In [17]:
!ngrok authtoken 2Oy4G6QRnRkrb0AomkVzibvTQmv_Qfd9mrNbPtRkygKjQtQ2

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000, timeout_keep_alive=3600)

INFO:     Started server process [1255]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://4975-34-87-3-8.ngrok-free.app
INFO:     117.219.22.193:0 - "GET / HTTP/1.1" 200 OK
